In this code

1. Extract eye gaze (X,Y) from video using computer vision.
2. Combine gaze data with existing simulator data for AI/ML analysis.

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

def load_data(filename):
 try:
   data = pd.read_csv(filename)
   return data
 except (FileNotFoundError, pd.errors.ParserError) as e:
   print("Error loading data:", e)
   return None

data = load_data("Experimenter_9110002_53.dat")

if data is None:
 exit()

In [ ]:
def get_time_diff(time_delta):  #Fxn to calculate time difference in seconds
 return time_delta / (10**7)

if data['LonAccel'].isnull().sum() > 0:
 print("Warning: 'LonAccel' column has missing values.")
if data['Velocity'].isnull().sum() > 0:
 print("Warning: 'Velocity' column has missing values.")

average_speed = data['Velocity'].mean()
speed_std = data['Velocity'].std()

print("Average Speed:", average_speed, "mph")
print("Standard Deviation of Speed:", speed_std, "mph")

harsh_acceleration = data[data['LonAccel'] > 3]
harsh_braking = data[data['LonAccel'] < -3]
sharp_turns = data[data['Steer'].abs() > 45]

print("Count of Harsh Acceleration Events:", len(harsh_acceleration))
print("Count of Harsh Braking Events:", len(harsh_braking))
print("Count of Sharp Turns:", len(sharp_turns))

lane_departures = data[data['LaneOffset'] > 0.5]
duration_offlane = lane_departures.groupby('FRAME_NUM')['DatTime'].diff().apply(get_time_diff).sum()

print("Total Duration Off Lane:", duration_offlane, "seconds")

data['Relative_Speed'] = data['Velocity'] - data['LeadVehVel'].fillna(0)
time_to_collision = data[data['Relative_Speed'] > 0]
time_to_collision['TTC'] = data['HeadwayDistance'] / data['Relative_Speed']

print("Minimum TTC:", time_to_collision['TTC'].min(), "seconds")
print("Average TTC:", time_to_collision['TTC'].mean(), "seconds")

In [ ]:
def plot_driver_input_heatmap(data):  #Visual Metrics
 time_stamps = data['DatTime'].apply(get_time_diff)
 steering = data['Steer']
 acceleration = data['LonAccel']
 braking = data['LonAccel'] * -1

 fig, ax = plt.subplots()
 ax.stackplot(time_stamps, [steering, acceleration, braking], labels=['Steering', 'Acceleration', 'Braking'])
 plt.xlabel('Time (s)')
 plt.ylabel('Input Intensity')
 plt.legend(title='Driver Input')
 plt.grid(True)
plt.show()